In [ ]:
import pandas as pd
import numpy as np

# Load excel data and store to dataframe

In [ ]:
# get into excel folder
#%cd drive/MyDrive/Colab\ Notebooks/pie_capstone

# Load the data
supplier_df = pd.read_excel('TTS calculation.xlsx', sheet_name="Supplier")
inventory_df = pd.read_excel('TTS calculation.xlsx', sheet_name="Inventory")
demand_df = pd.read_excel('TTS calculation.xlsx', sheet_name="Demand_Forecast")
#supply_df = pd.read_excel('TTS calculation.xlsx', sheet_name="Supply_Historical")
supply_split_df = pd.read_excel('TTS calculation.xlsx', sheet_name="Supplier_split")
region_df = pd.read_excel('TTS calculation.xlsx', sheet_name="Region")

In [ ]:
demand_df

In [ ]:
# round chemical lbs to nearest integer
supply_split_df[['Weekly Chemical Lbs']] = supply_split_df[['Weekly Chemical Lbs']]
inventory_df[['Inventory']] = inventory_df[['Inventory']]
inventory_df

# Aggregate, Combine, TTS

In [ ]:
# aggregate supply split data and combine it with inventory and demand
def prepare_combined_df(supply_df):
    df = supply_df.copy()

    # aggregate supply df
    df = df.groupby(['Chemical','Plant' ])['Weekly Chemical Lbs'].sum().reset_index()

    # combine inventory, demand and supply
    df = pd.merge(pd.merge(inventory_df, demand_df, on=['Key', 'Plant', 'Chemical']), df, on=['Plant', 'Chemical'], how='left')
    df = df.rename(columns={'Weekly Chemical Lbs': 'Weekly_Supply'})

    # Keep only necessary columns
    keep_cols = ['Chemical','Plant', 'Key', 'Inventory', 'Weekly_Demand', 'Weekly_Supply']

    # Replace nan to 0, meaning there is no supply
    df = df.fillna(0)

    return df.loc[:, keep_cols]

In [ ]:
# calculate tts based on combined table
def calculate_tts(combined_df):
    df = combined_df.copy()

    # Add column to df
    df['TTS'] = float("inf")

    # Calculate TTS for each row
    for i, row in df.iterrows():
        if row['Weekly_Supply'] >= row['Weekly_Demand']:
            continue
        else:
            tts = (row['Inventory'] / (row['Weekly_Demand'] - row['Weekly_Supply']));
            df.at[i, 'TTS'] = round(tts, 1)
    
    return df

### Example

In [ ]:
# Example
combined = prepare_combined_df(supply_split_df)
combined

In [ ]:
result = calculate_tts(combined)
result

# Original As Is Status


## Calculate TTS without distruption ➊

In [ ]:
tts_no_distruption = calculate_tts(prepare_combined_df(supply_split_df))
tts_no_distruption

### Plot bar chart

In [ ]:
grouped = tts_no_distruption.groupby(['Plant'])

df_TPVA = grouped.get_group('TPVA')
df_TPNM = grouped.get_group('TPNM')
df_TPIN = grouped.get_group('TPIN')

In [ ]:
inf_table_df = pd.DataFrame(columns=['Count of Infinity', 'Percentage(%)'], index=['TPVA', 'TPNM', 'TPIN'])

In [ ]:
def get_inf_count_and_percentage(plant_df):
    count = plant_df['TTS'].value_counts()[float("inf")]
    percentage = ((count / 51) * 100).round()

    return [count, percentage]


In [ ]:
# Insert values to df
inf_table_df.loc['TPVA'] = get_inf_count_and_percentage(df_TPVA)
inf_table_df.loc['TPNM'] = get_inf_count_and_percentage(df_TPNM)
inf_table_df.loc['TPIN'] = get_inf_count_and_percentage(df_TPIN)

inf_table_df

In [ ]:
# Plot each chart excluding TTS=inf
# Filter out TTS=inf for each dataframe
def filter_inf(df):
    return df.loc[df['TTS'] != float("inf")]

df_TPVA_filtered = filter_inf(df_TPVA)
df_TPNM_filtered = filter_inf(df_TPNM)
df_TPIN_filtered = filter_inf(df_TPIN)


In [ ]:
# import matplotlib.pyplot as plt

# #Set up subplots
# fig, axs = plt.subplots(1, 3, figsize=(12, 6))

# df_TPVA_filtered.sort_values("TTS", ascending=True).plot.barh(x='Chemical', y='TTS', title="TPVA", ax=axs[0])
# axs[0].set_ylabel("") # remove y-axis label

# df_TPNM_filtered.sort_values("TTS", ascending=True).plot.barh(x='Chemical', y='TTS', title="TPNM", ax=axs[1])
# axs[1].set_ylabel("") # remove y-axis label

# df_TPIN_filtered.sort_values("TTS", ascending=True).plot.barh(x='Chemical', y='TTS', title="TPIN", ax=axs[2])
# axs[2].set_ylabel("") # remove y-axis label


# fig.suptitle("TPVA, TPNM, and TPIN")

# #Adjust spacing between subplots
# plt.subplots_adjust(wspace=0.7)
# plt.show()


## Calculate TTS after turning off each supplier

In [ ]:
supplier_df

### Function: turn off supplier

In [ ]:
# turn off supplier
def turn_off_supplier(sup, supply_df):
    df = supply_df.copy()

    for i, row in df.iterrows():
        if row['Supplier'] == sup:
            df.at[i, 'Weekly Chemical Lbs'] = 0
        
    return df

In [ ]:
test = turn_off_supplier('Swan', supply_split_df)
test

### Result without stretching suppliers

In [ ]:
# function to get turned of tts and add to our result df
def add_tts_to_result(sup, off_tts_df, result):
    # suffix string for display
    suffix_str = '_' + sup + '_off'

    # merge current result df with turned off supplier df calculated
    return result.merge(off_tts_df[['Chemical','Plant', 'TTS']].set_index(['Chemical', 'Plant']).add_suffix(suffix_str), on=['Chemical', 'Plant'])

In [ ]:
# Generate the result table

# Use the result of no distruption as base
result_df = tts_no_distruption[['Chemical', 'Plant', 'TTS']]

# loop through each supplier and add turned off tts column to our result
for i, row in supplier_df.iterrows():
    sup = row['Supplier']
    
    # get tts df with supplier turned off
    off_df = turn_off_supplier(sup, supply_split_df)
    off_tts_df = calculate_tts(prepare_combined_df(off_df))

    # add tts to result df
    result_df = add_tts_to_result(sup, off_tts_df, result_df)

result_df

### Result with stretching 20%


In [ ]:
# stretch suppliers to 20%
def get_stretched_supply_df(sup, supply_df):
    off_df = turn_off_supplier(sup, supply_df)

    # stretch other suppliers
    # find all chemicals provided by cur turned off supplier
    sup_chemicals = off_df.loc[off_df['Supplier'] == sup].drop_duplicates(subset=['Chemical'])['Chemical'].tolist()
    
    # all chemicals' all suppliers x 1.2
    for i, row in off_df.iterrows():
        if row['Chemical'] in sup_chemicals:
            off_df.at[i, 'Weekly Chemical Lbs'] = (row['Weekly Chemical Lbs'] * 1.2)

    return off_df

# stretched = get_stretched_supply_df('Swan', supply_split_df)
# stretched

In [ ]:
# Generate the result table

# Use the result of no distruption as base
stretched_result_df = tts_no_distruption[['Chemical', 'Plant', 'TTS']]

# loop through each supplier and add turned off tts column to our result
for i, row in supplier_df.iterrows():
    sup = row['Supplier']
    
    # get tts df with supplier turned off
    off_df = get_stretched_supply_df(sup, supply_split_df)
    off_tts_df = calculate_tts(prepare_combined_df(off_df))

    # add tts to result df
    stretched_result_df = add_tts_to_result(sup, off_tts_df, stretched_result_df)

stretched_result_df

### Final Result table

In [ ]:
final_result_sup_df = result_df.merge(stretched_result_df.drop('TTS', axis=1).set_index(['Chemical', 'Plant']).add_suffix('_stretched'), on=['Chemical', 'Plant'])
final_result_sup_df

## Calculate TTS after turning off each region

In [ ]:
region_df

### Function: turn off region

In [ ]:
# turn off region
def turn_off_region(region, supply_df):
    df = supply_df.copy()

    for i, row in df.iterrows():
        if row['Region'] == region:
            df.at[i, 'Weekly Chemical Lbs'] = 0
        
    return df

In [ ]:
test = turn_off_region('Texas', supply_split_df)
test

### Result without stretching suppliers

In [ ]:
# Generate the result table

# Use the result of no distruption as base
region_result_df = tts_no_distruption[['Chemical', 'Plant', 'TTS']]

# loop through each region and add turned off tts column to our result
for i, row in region_df.iterrows():
    region = row['Region_group']
    
    # get tts df with region turned off
    off_df = turn_off_region(region, supply_split_df)
    off_tts_df = calculate_tts(prepare_combined_df(off_df))

    # add tts to result df
    region_result_df = add_tts_to_result(region, off_tts_df, region_result_df)

region_result_df

### Result with stretching 20%

In [ ]:
# stretch suppliers to 20%
def get_stretched_region_df(region, supply_df):
    off_df = turn_off_region(region, supply_df)

    # stretch other regions
    # find all chemicals provided by cur turned off region
    sup_chemicals = off_df.loc[off_df['Region'] == region].drop_duplicates(subset=['Chemical'])['Chemical'].tolist()
    
    # all chemicals' all regions x 1.2
    for i, row in off_df.iterrows():
        if row['Chemical'] in sup_chemicals:
            off_df.at[i, 'Weekly Chemical Lbs'] = (row['Weekly Chemical Lbs'] * 1.2)

    return off_df

In [ ]:
# Generate the result table

# Use the result of no distruption as base
stretched_region_result_df = tts_no_distruption[['Chemical', 'Plant', 'TTS']]

# loop through each region and add turned off tts column to our result
for i, row in region_df.iterrows():
    region = row['Region_group']
    
    # get tts df with region turned off
    off_df = get_stretched_region_df(region, supply_split_df)
    off_tts_df = calculate_tts(prepare_combined_df(off_df))

    # add tts to result df
    stretched_region_result_df = add_tts_to_result(region, off_tts_df, stretched_region_result_df)

stretched_region_result_df

### Final Result Table

In [ ]:
final_result_region_df = region_result_df.merge(stretched_region_result_df.drop('TTS', axis=1).set_index(['Chemical', 'Plant']).add_suffix('_stretched'), on=['Chemical', 'Plant'])
final_result_region_df

# Optimized Scenario


## Transfer table (Data Cleaning)

In [ ]:
# Load the data
optimized_supply_df = pd.read_excel('optimization_result.xlsx')

# Load the mapping table
region_map_df = pd.read_excel('TTS calculation.xlsx', sheet_name="Region_mapping")
supplier_map_df = pd.read_excel('TTS calculation.xlsx', sheet_name="Supplier_mapping")

In [ ]:
optimized_supply_df = optimized_supply_df.rename(columns={"Material": "Chemical", "Quantity": "Chemical Lbs", "Supplier": "Supplier_dummy"})
optimized_supply_df

In [ ]:
# Update Supplier name 
optimized_supply_df['Supplier'] = optimized_supply_df['Supplier_dummy'].map(supplier_map_df.set_index('Supplier#')['Supplier.1'])
optimized_supply_df

In [ ]:
region_map_df = region_map_df[region_map_df['Supplier Locations'].notna()]
region_map_df = region_map_df.set_index('Supplier Locations')

In [ ]:
# Update Region name
optimized_supply_df['Region'] = optimized_supply_df['Location'].map(region_map_df['Region_v2_suggestion'])
optimized_supply_df

In [ ]:
# Add Weekly Chemical Pounds
optimized_supply_df['Weekly Chemical Lbs'] = (optimized_supply_df['Chemical Lbs'] / 50).apply(np.ceil)
optimized_supply_df

## Calculate TTS without distruption

In [ ]:
opt_tts_no_distruption = calculate_tts(prepare_combined_df(optimized_supply_df))
opt_tts_no_distruption

### Plot bar chart

In [ ]:
opt_grouped = opt_tts_no_distruption.groupby(['Plant'])

df_TPVA_opt = opt_grouped.get_group('TPVA')
df_TPNM_opt = opt_grouped.get_group('TPNM')
df_TPIN_opt = opt_grouped.get_group('TPIN')

In [ ]:
opt_inf_table_df = pd.DataFrame(columns=['Count of Infinity', 'Percentage(%)'], index=['TPVA', 'TPNM', 'TPIN'])

In [ ]:
def get_inf_count_and_percentage(plant_df):
    count = plant_df['TTS'].value_counts()[float("inf")]
    percentage = ((count / 51) * 100).round()

    return [count, percentage]


In [ ]:
# Insert values to df
opt_inf_table_df.loc['TPVA'] = get_inf_count_and_percentage(df_TPVA_opt)
opt_inf_table_df.loc['TPNM'] = get_inf_count_and_percentage(df_TPNM_opt)
opt_inf_table_df.loc['TPIN'] = get_inf_count_and_percentage(df_TPIN_opt)

opt_inf_table_df

In [ ]:
# Plot each chart excluding TTS=inf
# Filter out TTS=inf for each dataframe
def filter_inf(df):
    return df.loc[df['TTS'] != float("inf")]

opt_df_TPVA_filtered = filter_inf(df_TPVA_opt)
opt_df_TPNM_filtered = filter_inf(df_TPNM_opt)
opt_df_TPIN_filtered = filter_inf(df_TPIN_opt)


In [ ]:
opt_df_TPNM_filtered

In [ ]:
# import matplotlib.pyplot as plt

# # Set up subplots
# fig, axs = plt.subplots(1, 3, figsize=(12, 6))

# #opt_df_TPVA_filtered.sort_values("TTS", ascending=True).plot.barh(x='Chemical', y='TTS', title="TPVA", ax=axs[0])
# axs[0].set_ylabel("") # remove y-axis label

# opt_df_TPNM_filtered.sort_values("TTS", ascending=True).plot.barh(x='Chemical', y='TTS', title="TPNM", ax=axs[1])
# axs[1].set_ylabel("") # remove y-axis label

# #opt_df_TPIN_filtered.sort_values("TTS", ascending=True).plot.barh(x='Chemical', y='TTS', title="TPIN", ax=axs[2])
# axs[2].set_ylabel("") # remove y-axis label


# fig.suptitle("TPVA, TPNM, and TPIN")

# # Adjust spacing between subplots
# plt.subplots_adjust(wspace=0.7)
# plt.show()


## Calculate TTS after turning off each supplier

### Result without stretching suppliers

In [ ]:
# Generate the result table

# Use the result of no distruption as base
opt_result_df = tts_no_distruption[['Chemical', 'Plant', 'TTS']]

# loop through each supplier and add turned off tts column to our result
for i, row in supplier_df.iterrows():
    sup = row['Supplier']
    
    # get tts df with supplier turned off
    off_df = turn_off_supplier(sup, optimized_supply_df)
    off_tts_df = calculate_tts(prepare_combined_df(off_df))

    # add tts to result df
    opt_result_df = add_tts_to_result(sup, off_tts_df, opt_result_df)

opt_result_df

### Result with stretching 20%

In [ ]:
# Generate the result table

# Use the result of no distruption as base
opt_stretched_result_df = tts_no_distruption[['Chemical', 'Plant', 'TTS']]

# loop through each supplier and add turned off tts column to our result
for i, row in supplier_df.iterrows():
    sup = row['Supplier']
    
    # get tts df with supplier turned off
    off_df = get_stretched_supply_df(sup, optimized_supply_df)
    off_tts_df = calculate_tts(prepare_combined_df(off_df))

    # add tts to result df
    opt_stretched_result_df = add_tts_to_result(sup, off_tts_df, opt_stretched_result_df)

opt_stretched_result_df

### Final Result Table

In [ ]:
opt_final_result_sup_df = opt_result_df.merge(opt_stretched_result_df.drop('TTS', axis=1).set_index(['Chemical', 'Plant']).add_suffix('_stretched'), on=['Chemical', 'Plant'])
opt_final_result_sup_df

## Calculate TTS after turning off each region

### Result without stretching suppliers

In [ ]:
# Generate the result table

# Use the result of no distruption as base
opt_region_result_df = tts_no_distruption[['Chemical', 'Plant', 'TTS']]

# loop through each region and add turned off tts column to our result
for i, row in region_df.iterrows():
    region = row['Region_group']
    
    # get tts df with region turned off
    off_df = turn_off_region(region, optimized_supply_df)
    off_tts_df = calculate_tts(prepare_combined_df(off_df))

    # add tts to result df
    opt_region_result_df = add_tts_to_result(region, off_tts_df, opt_region_result_df)

opt_region_result_df

### Result with stretching 20%

In [ ]:
# Generate the result table

# Use the result of no distruption as base
opt_stretched_region_result_df = tts_no_distruption[['Chemical', 'Plant', 'TTS']]

# loop through each region and add turned off tts column to our result
for i, row in region_df.iterrows():
    region = row['Region_group']
    
    # get tts df with region turned off
    off_df = get_stretched_region_df(region, optimized_supply_df)
    off_tts_df = calculate_tts(prepare_combined_df(off_df))

    # add tts to result df
    opt_stretched_region_result_df = add_tts_to_result(region, off_tts_df, opt_stretched_region_result_df)

opt_stretched_region_result_df

### Final Result Table

In [ ]:
opt_final_result_region_df = opt_region_result_df.merge(opt_stretched_region_result_df.drop('TTS', axis=1).set_index(['Chemical', 'Plant']).add_suffix('_stretched'), on=['Chemical', 'Plant'])
opt_final_result_region_df

# Save the result into excel file

In [ ]:
# Original result turning off supplier #final_result_sup_df
# Original result turning off region #final_result_region_df
# Optimized result turning off supplier #opt_final_result_sup_df
# Optimized result turning off region #opt_final_result_region_df


# Saved all the result into excel file
# create a excel writer object
with pd.ExcelWriter("TTS_combined_result.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    final_result_sup_df.to_excel(writer, sheet_name="final_result_sup", index=False)
    final_result_region_df.to_excel(writer, sheet_name="final_result_region", index=False)
    opt_final_result_sup_df.to_excel(writer, sheet_name="opt_final_result_sup", index=False)
    opt_final_result_region_df.to_excel(writer, sheet_name="opt_final_result_region", index=False)

# Summary

## Supplier Summary

In [ ]:
# result_df
# stretched_result_df
# opt_result_df
# opt_stretched_result_df

In [ ]:
# create empty dataframe with columns
supplier_summary_df = pd.DataFrame(columns=['Chemical', 'Plant', 'TTS without stretch', 'TTS without stretch supplier', 'TTS with stretch', 'TTS with stretch supplier', 'Optimized TTS without stretch', 'Optimized TTS without stretch supplier', 'Optimized TTS with stretch', 'Optimized TTS with stretch supplier'])

supplier_summary_df['Chemical'] = result_df['Chemical'].copy()
supplier_summary_df['Plant'] = result_df['Plant'].copy()
supplier_summary_df

In [ ]:
# find minimum TTS
# from result_df
supplier_summary_df['TTS without stretch'] = result_df.drop(['Chemical', 'Plant'], axis=1).min(axis='columns')
supplier_summary_df['TTS without stretch supplier'] = result_df.drop(['Chemical', 'Plant'], axis=1).idxmin(axis='columns')

# from stretched_result_df
supplier_summary_df['TTS with stretch'] = stretched_result_df.drop(['Chemical', 'Plant'], axis=1).min(axis='columns')
supplier_summary_df['TTS with stretch supplier'] = stretched_result_df.drop(['Chemical', 'Plant'], axis=1).idxmin(axis='columns')

# from opt_result_df
supplier_summary_df['Optimized TTS without stretch'] = opt_result_df.drop(['Chemical', 'Plant'], axis=1).min(axis='columns')
supplier_summary_df['Optimized TTS without stretch supplier'] = opt_result_df.drop(['Chemical', 'Plant'], axis=1).idxmin(axis='columns')

# opt_stretched_result_df
supplier_summary_df['Optimized TTS with stretch'] = opt_stretched_result_df.drop(['Chemical', 'Plant'], axis=1).min(axis='columns')
supplier_summary_df['Optimized TTS with stretch supplier'] = opt_stretched_result_df.drop(['Chemical', 'Plant'], axis=1).idxmin(axis='columns')

supplier_summary_df

## Region Summary

In [ ]:
# region_result_df
# stretched_region_result_df
# opt_region_result_df
# opt_stretched_region_result_df

In [ ]:
# create empty dataframe with columns
region_summary_df = pd.DataFrame(columns=['Chemical', 'Plant', 'TTS without stretch', 'TTS without stretch region', 'TTS with stretch', 'TTS with stretch region', 'Optimized TTS without stretch', 'Optimized TTS without stretch region', 'Optimized TTS with stretch', 'Optimized TTS with stretch region'])

region_summary_df['Chemical'] = result_df['Chemical'].copy()
region_summary_df['Plant'] = result_df['Plant'].copy()
region_summary_df

In [ ]:
# find minimum TTS
# from region_result_df
region_summary_df['TTS without stretch'] = region_result_df.drop(['Chemical', 'Plant'], axis=1).min(axis='columns')
region_summary_df['TTS without stretch region'] = region_result_df.drop(['Chemical', 'Plant'], axis=1).idxmin(axis='columns')

# from stretched_region_result_df
region_summary_df['TTS with stretch'] = stretched_region_result_df.drop(['Chemical', 'Plant'], axis=1).min(axis='columns')
region_summary_df['TTS with stretch region'] = stretched_region_result_df.drop(['Chemical', 'Plant'], axis=1).idxmin(axis='columns')

# from opt_region_result_df
region_summary_df['Optimized TTS without stretch'] = opt_region_result_df.drop(['Chemical', 'Plant'], axis=1).min(axis='columns')
region_summary_df['Optimized TTS without stretch region'] = opt_region_result_df.drop(['Chemical', 'Plant'], axis=1).idxmin(axis='columns')

# opt_stretched_region_result_df
region_summary_df['Optimized TTS with stretch'] = opt_stretched_region_result_df.drop(['Chemical', 'Plant'], axis=1).min(axis='columns')
region_summary_df['Optimized TTS with stretch region'] = opt_stretched_region_result_df.drop(['Chemical', 'Plant'], axis=1).idxmin(axis='columns')

region_summary_df

## Save the result into excel file

In [ ]:
# Saved 2 tables into excel file
# create a excel writer object
with pd.ExcelWriter("TTS_combined_summary.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    supplier_summary_df.to_excel(writer, sheet_name="supplier_summary", index=False)
    region_summary_df.to_excel(writer, sheet_name="region_summary", index=False)